# **Prompt Generation**

In this notebook is illustrated how to generate prompts by using the implemented libraries.

Note that these prompts are essentially designed for GPT models, or for those supporting the following roles:
- **system**,  which provides high-level instructions to guide model’s behavior throughout the conversation;
- **user**, it is the model’s response, usually simulated when using few-shot prompting;
- **assistant**, that presents queries related to the task the LLM is asked to perform.

In [1]:
import os
import sys
sys.path.append('\\'.join(os.getcwd().split('\\')[:-1])+'\\src')

from src.data.TFQuestionnairesDataset import TFQuestionnairesDataset
from src.prompts.PromptGenerator import PromptGenerator

from pprint import pprint

## **Questionnaire generation**

The first step of this thesis work is the questionnaire generaion. The designed prompts are reported below.

In [2]:
# ----------------
# Load data
# ----------------
dataset = TFQuestionnairesDataset()
dataset.load_data(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

samples = dataset.get_sample_questionnaire_data()
questionnaire_id = samples.questionnaires["ID"]

question_types = samples.get_question_types()

### *System prompt*

The **system** prompt has two variants according to the given specifications:
1. with all parameters: topic, number and type of questions to be generated
2. with only the questionnaire topic

In [3]:
system_generator = PromptGenerator("system")

with_all_params = system_generator.generate_prompt(has_full_params=True, topic=samples.questionnaires["NAME"], question_type="Single choice", question_number=5, 
                                                   question_types_data=question_types)

with_only_topic = system_generator.generate_prompt(has_full_params=False, topic=samples.questionnaires["NAME"], question_types_data=question_types)

Both of them are structured as follows:
1. **Role definition**, used to define the role the LLM has to impersonate;
2. **Input definition**, which define the information that the user can specify and (eventually) the used format;
3. **Question types**, used to define the admissible question types;
4. **Task definition**, which is used to instruct the LLM on the task it has to perform;
5. **Output definition**, it constrates the LLM to use the specified format while generating its response.

In [4]:
pprint(with_all_params)

('You are a Questionnaire Generator and you work in the Human Resources '
 'Management field.The user will ask you to generate a questionnaire '
 'specifying:\n'
 '    - topic\n'
 '    - number of questions\n'
 '    - type of questions\n'
 'If the user does not specify a valid topic, reply with "Sorry I cant help '
 'you".If the topic is valid, reply with only a JSON, which must respect the '
 'following format:Your output must be a JSON which must respect the following '
 'format:\n'
 '    - The root of the JSON is an object that contains a single property '
 "'data'.\n"
 "    - The 'data' property is an object that contains a single property "
 "'TF_QUESTIONNAIRES'.\n"
 "    - 'TF_QUESTIONNAIRES' is an array of objects, each representing a "
 'questionnaire. \n'
 '    - Each questionnaire object has the following properties:\n'
 "        - 'CODE': A string representing the code of the questionnaire.\n"
 "        - 'NAME': A string representing the name of the questionnaire.\n"
 "    

In [5]:
pprint(with_only_topic)

('You are a Questionnaire Generator and you work in the Human Resources '
 'Management field.The user will ask you to generate a questionnaire about a '
 'specified topic. \n'
 'Deciding the proper number and type of questions is up to you.If the user '
 'does not specify a valid topic, reply with "Sorry I cant help you".If the '
 'topic is valid, reply with only a JSON, which must respect the following '
 'format:Your output must be a JSON which must respect the following format:\n'
 '    - The root of the JSON is an object that contains a single property '
 "'data'.\n"
 "    - The 'data' property is an object that contains a single property "
 "'TF_QUESTIONNAIRES'.\n"
 "    - 'TF_QUESTIONNAIRES' is an array of objects, each representing a "
 'questionnaire. \n'
 '    - Each questionnaire object has the following properties:\n'
 "        - 'CODE': A string representing the code of the questionnaire.\n"
 "        - 'NAME': A string representing the name of the questionnaire.\n"
 "     

### *Assistant prompt*

The **assistant** prompt represents the LLM's output. It's used only when using the *few-shot* prompting technique.

In [6]:
assistant_generator = PromptGenerator("assistant")

formatted_json = dataset.to_json(questionnaire_id)

response = assistant_generator.generate_prompt(json=formatted_json)
pprint(response)

('{"data": {"TF_QUESTIONNAIRES": [{"CODE": "Stress Survey", "NAME": "Stress '
 'Survey", "_TF_QUESTIONS": [{"CODE": "1", "NAME": "I often feel overwhelmed '
 'by tasks which I\'m not sure to complete in time", "TYPE_ID": 3, '
 '"DISPLAY_ORDER": 1, "_TF_ANSWERS": [{"ANSWER": "Completely Disagree"}, '
 '{"ANSWER": "2"}, {"ANSWER": "3"}, {"ANSWER": "4"}, {"ANSWER": "5"}, '
 '{"ANSWER": "6"}, {"ANSWER": "7"}, {"ANSWER": "8"}, {"ANSWER": "9"}, '
 '{"ANSWER": "Strongly Agree"}, {"ANSWER": "Completely Disagree"}, {"ANSWER": '
 '"2"}, {"ANSWER": "3"}, {"ANSWER": "4"}, {"ANSWER": "5"}, {"ANSWER": "6"}, '
 '{"ANSWER": "7"}, {"ANSWER": "8"}, {"ANSWER": "9"}, {"ANSWER": "Strongly '
 'Agree"}, {"ANSWER": "Completely Disagree"}, {"ANSWER": "2"}, {"ANSWER": '
 '"3"}, {"ANSWER": "4"}, {"ANSWER": "5"}, {"ANSWER": "6"}, {"ANSWER": "7"}, '
 '{"ANSWER": "8"}, {"ANSWER": "9"}, {"ANSWER": "Strongly Agree"}, {"ANSWER": '
 '"Completely Disagree"}, {"ANSWER": "2"}, {"ANSWER": "3"}, {"ANSWER": "4"}, '
 '{"ANSWE

### *User prompt*

The **user** prompt reprents the user request. Similarly to the *system* prompt, it has two variants.

In [7]:
user_generator = PromptGenerator("user")

with_all_params = user_generator.generate_prompt(has_full_params=True, topic=samples.questionnaires["NAME"], question_type="Single choice", question_number=5)
with_only_topic = user_generator.generate_prompt(has_full_params=False, topic=samples.questionnaires["NAME"])

In [8]:
pprint(with_all_params)

'Generate me a questionnaire on Stress Survey with 5 Single choice questions'


In [9]:
pprint(with_only_topic)

'Generate me a questionnaire on Stress Survey'
